In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
dataset_dir = 'dataset/original_data'

datasets = os.listdir(dataset_dir)
datasets_df = pd.DataFrame({'filenames': datasets})

In [3]:
datasets_df['filenames']

0           subjecta-neutral-1.csv
1     subjectd-concentrating-1.csv
2           subjectc-neutral-2.csv
3           subjectd-relaxed-2.csv
4           subjectb-relaxed-1.csv
5           subjectb-relaxed-2.csv
6           subjectd-relaxed-1.csv
7     subjectd-concentrating-2.csv
8     subjecta-concentrating-2.csv
9     subjectb-concentrating-1.csv
10    subjectc-concentrating-2.csv
11        name-concentrating-1.csv
12          subjectc-neutral-1.csv
13          subjecta-relaxed-2.csv
14          subjectd-neutral-2.csv
15          subjecta-neutral-2.csv
16          subjectc-relaxed-2.csv
17          subjectb-neutral-2.csv
18    subjectb-concentrating-2.csv
19          subjectb-neutral-1.csv
20          subjecta-relaxed-1.csv
21    subjectc-concentrating-1.csv
22          subjectd-neutral-1.csv
23          subjectc-relaxed-1.csv
24    subjecta-concentrating-1.csv
Name: filenames, dtype: object

In [4]:
classes = ['concentrating', 'neutral']#, 'relaxed']
datasets_by_class = {'concentrating':[], 'neutral':[], 'relaxed':[]}
window_size = 256 #samples


for class_name in classes:
    datasets = datasets_df[datasets_df['filenames'].str.contains(class_name)]

    for filename in datasets['filenames']:
        sub_df = pd.read_csv(f"{dataset_dir}/{filename}") 

        windows = []
        cur_window = []
        sample_count = 0

        for row in range(sub_df.shape[0]):
            sample_count+=1
            cur_window.append(sub_df.iloc[row])

            if sample_count == window_size:
                windows.append(cur_window)
                cur_window = []
                sample_count = 0

        # datasets_by_class[class_name].append(np.array(windows))
        datasets_by_class[class_name]+=windows

In [5]:
concentrating_data = np.asarray(datasets_by_class['concentrating'], dtype='object')
neutral_data = np.asarray(datasets_by_class['neutral'], dtype='object')
# relaxed_data = np.asarray(datasets_by_class['relaxed'], dtype='object')

In [6]:
print(concentrating_data.shape)
print(neutral_data.shape)
# print(relaxed_data.shape)

(423, 256, 6)
(422, 256, 6)


In [7]:
matrix = concentrating_data[0][:,1]
print(matrix.shape)

(256,)


In [8]:
# One sample
concentrating_data[0]

array([[1533223253.545, 11.719, 25.879, -240.234, 24.902, 70.801],
       [1533223253.549, 9.766, 24.414, -465.82, 18.555, 64.941],
       [1533223253.553, 13.672, 29.785, -401.367, 10.254, 33.691],
       ...,
       [1533223254.533, 15.625, 27.344, 254.883, 14.648, 58.594],
       [1533223254.537, 12.207, 28.809, 340.82, 19.531, 18.555],
       [1533223254.541, 7.812, 30.273, 333.008, 18.066, -30.762]],
      dtype=object)

In [9]:
X = concentrating_data[:, :, 1:]  # Remove the first column
X = np.concatenate([X, neutral_data[:, :, 1:]], axis=0)

y = np.array([0] * concentrating_data[:, :, 1:].shape[0] + [1] * neutral_data[:, :, 1:].shape[0])


In [10]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

X, y = shuffle(X, y, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [11]:
print(X_train.shape)
print(y_train.shape)

(676, 256, 5)
(676,)


In [12]:
def transform_set(scalar, data, fit=False):
    original_shape = data.shape 
    reshaped = data.reshape(-1, data.shape[-1]) 

    if fit:
        reshaped = scalar.fit_transform(reshaped)
    else:
        reshaped = scalar.transform(reshaped)
        
    reshaped = reshaped.reshape(original_shape)

    return reshaped

In [13]:
#Normalize the data
from sklearn.preprocessing import MinMaxScaler
X_scalar = MinMaxScaler()
X_train_scaled = transform_set(X_scalar, X_train, fit=True)
X_test_scaled = transform_set(X_scalar, X_test, fit=False)

In [14]:
print("X data:", X_train.shape, X_test.shape)
print("y data:", y_train.shape, y_test.shape)

X data: (676, 256, 5) (169, 256, 5)
y data: (676,) (169,)


In [15]:
from tensorflow import keras
from tensorflow.keras import layers

2025-03-15 11:32:12.995833: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742063533.020385   11607 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742063533.024786   11607 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742063533.037864   11607 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742063533.037906   11607 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742063533.037909   11607 computation_placer.cc:177] computation placer alr

In [21]:
# Define the input shape (256x5)
input_shape = (256, 5) 

# Build the CNN model
model = keras.Sequential([
    layers.Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(64, kernel_size=3, activation='relu'),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(128, kernel_size=3, activation='relu'),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Two output neurons for two classes
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Display the model summary
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_3 (Conv1D)               │ (None, 254, 32)        │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 127, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 125, 64)        │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 62, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ (None, 60, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 7680)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       983,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,014,721 (3.87 MB)

 Trainable params: 1,014,721 (3.87 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(X_train_scaled, y_train, epochs=80, validation_data=(X_test_scaled, y_test))

Epoch 1/16
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9563 - loss: 0.1085 - val_accuracy: 0.9112 - val_loss: 0.5192
Epoch 2/16
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9423 - loss: 0.1316 - val_accuracy: 0.8757 - val_loss: 0.5003
Epoch 3/16
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.8715 - loss: 0.3706 - val_accuracy: 0.5680 - val_loss: 1.2285
Epoch 4/16
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7763 - loss: 0.4807 - val_accuracy: 0.8935 - val_loss: 0.4102
Epoch 5/16
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9109 - loss: 0.2650 - val_accuracy: 0.8876 - val_loss: 0.4289
Epoch 6/16
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9530 - loss: 0.1484 - val_accuracy: 0.8876 - val_loss: 0.4049
Epoch 7/16
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9775 - loss: 0.0804 - val_accuracy: 0.8935 - val_loss: 0.4133
Epoch 8/16
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9773 - loss: 0.0679 - val_accuracy: 0.9172 - val_lo

In [18]:
X_train_scaled[0].shape
X_test_scaled[0].shape

(256, 5)

In [19]:
preds = model.predict(X_test_scaled)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step


In [20]:
score = 0

for i in range(preds.shape[0]):
    is_equal = (np.round(preds[i]) == np.round(y_test[i]))
    if is_equal:
        score+=1

print(f"{score}/{preds.shape[0]}")

145/169
